<h1>Table of Contents &lt;br&gt;&lt;/br&gt;<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Bulk-Data-Merit-Order" data-toc-modified-id="Bulk-Data-Merit-Order-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Bulk Data Merit Order</a></span><ul class="toc-item"><li><span><a href="#List-of-Tenders-(Ausschreibung)" data-toc-modified-id="List-of-Tenders-(Ausschreibung)-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>List of Tenders (Ausschreibung)</a></span></li><li><span><a href="#Result-of-Tenders-(Abgegebene-Angebote)" data-toc-modified-id="Result-of-Tenders-(Abgegebene-Angebote)-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Result of Tenders (Abgegebene Angebote)</a></span></li><li><span><a href="#Activated-Control-Reserve" data-toc-modified-id="Activated-Control-Reserve-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Activated Control Reserve</a></span></li></ul></li><li><span><a href="#Balancing-Market-(regelleistungen)" data-toc-modified-id="Balancing-Market-(regelleistungen)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Balancing Market (regelleistungen)</a></span></li><li><span><a href="#Calculate-Merit-Order" data-toc-modified-id="Calculate-Merit-Order-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Calculate Merit Order</a></span></li></ul></div>

In [2]:
# Display plots inline
%matplotlib inline

# Autoreload all package before excecuting a call
%load_ext autoreload
%autoreload 2

In [3]:
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Bulk Data Merit Order

## List of Tenders (Ausschreibung)

In [231]:
df_tenders = pd.read_csv("../data/raw/tenders_2016_2017.csv", sep=';', index_col=False,
                     dayfirst=True, parse_dates=[0, 1, 2, 3,4], infer_datetime_format=True, decimal=',')
df_tenders = df_tenders[['DATE_FROM', 'DATE_TO', 'GATE_OPEN_TIME', 'GATE_COSURE_TIME', 'PRODUCT', 'TOTAL_DEMAND_[MW]']]
df_tenders.columns = ['from', 'to', 'gate_opening', 'gate_closure', 'product', 'demand_mw']
df_tenders.head(20)

,from,to,gate_opening,gate_closure,product,demand_mw
0,2015-12-28,2016-01-03,2015-12-18 12:00:00,2015-12-22 15:00:00,NEG_HT,2500
1,2015-12-28,2016-01-03,2015-12-18 12:00:00,2015-12-22 15:00:00,NEG_NT,2500
2,2015-12-28,2016-01-03,2015-12-18 12:00:00,2015-12-22 15:00:00,POS_HT,2500
3,2015-12-28,2016-01-03,2015-12-18 12:00:00,2015-12-22 15:00:00,POS_NT,2500
4,2016-01-04,2016-01-10,2015-12-23 12:00:00,2015-12-29 15:00:00,NEG_HT,1979
5,2016-01-04,2016-01-10,2015-12-23 12:00:00,2015-12-29 15:00:00,NEG_NT,1979
6,2016-01-04,2016-01-10,2015-12-23 12:00:00,2015-12-29 15:00:00,POS_HT,2054
7,2016-01-04,2016-01-10,2015-12-23 12:00:00,2015-12-29 15:00:00,POS_NT,2054
8,2016-01-11,2016-01-17,2015-12-30 12:00:00,2016-01-05 15:00:00,NEG_HT,1979
9,2016-01-11,2016-01-17,2015-12-30 12:00:00,2016-01-05 15:00:00,NEG_NT,1979


## Result of Tenders (Abgegebene Angebote)

In [230]:
df_results = pd.read_csv("../data/raw/results_2016_2017.csv", sep=';', index_col=False,
                     dayfirst=True, parse_dates=[0, 1], infer_datetime_format=True, decimal=',')
df_results.drop(['TYPE_OF_RESERVES', 'COUNTRY'], inplace=True, axis=1)
df_results.columns = ['from', 'to', 'product', 'capacity_price_mw','energy_price_mwh', 'payment_direction', 'offered_mw', 'allocated_mw']
df_results['payment_direction'].replace(['GRID_TO_PROVIDER', 'PROVIDER_TO_GRID'], ['GP', 'PG'], inplace=True)
df_results.head(20)

,from,to,product,capacity_price_mw,energy_price_mwh,payment_direction,offered_mw,allocated_mw
0,2015-12-28,2016-01-03,NEG_HT,589.0,500.0,GP,5,5
1,2015-12-28,2016-01-03,NEG_HT,689.0,5999.0,GP,5,5
2,2015-12-28,2016-01-03,NEG_HT,645.0,400.0,GP,5,5
3,2015-12-28,2016-01-03,NEG_HT,645.0,400.0,GP,5,5
4,2015-12-28,2016-01-03,NEG_HT,565.0,4440.0,GP,8,8
5,2015-12-28,2016-01-03,NEG_HT,580.0,4440.0,GP,8,8
6,2015-12-28,2016-01-03,NEG_HT,635.0,4440.0,GP,6,6
7,2015-12-28,2016-01-03,NEG_HT,580.0,4440.0,GP,8,8
8,2015-12-28,2016-01-03,NEG_HT,580.0,4440.0,GP,5,5
9,2015-12-28,2016-01-03,NEG_HT,440.0,1500.0,GP,9,9


## Activated Control Reserve

In [225]:
df_activated = pd.read_csv("../data/raw/balancing/activated_secondary_reserve_2016_2017.csv", sep=';', decimal=',', thousands='.', index_col=False, dayfirst=True, parse_dates=[0], infer_datetime_format=True)
df_activated.drop(['LETZTE AENDERUNG', 'ERSATZWERT','LETZTE AENDERUNG.1', 'QUAL. NEG', 'QUAL. POS'], axis=1, inplace=True)
df_activated.columns = ['date', 'from', 'to', 'neg_mw', 'pos_mw']
#print(df_activated)
hours_minutes_from = df_activated['from'].str.split(":", expand=True)
df_activated['from'] = pd.to_datetime(df_activated['date'].astype(str) + " " + hours_minutes_from[0] + ":" + hours_minutes_from[1])

hours_minutes_to = df_activated['to'].str.split(":", expand=True)
df_activated['to'] = pd.to_datetime(df_activated['date'].astype(str) + " " + hours_minutes_to[0] + ":" + hours_minutes_to[1])

# Fix time where 0:00 belongs to previous day
df_activated.loc[(df_activated['to'].dt.hour == 0) & (df_activated['to'].dt.minute == 0), 'to'] = df_activated.to + pd.DateOffset(days=1)

df_activated.drop('date', inplace=True, axis=1)
df_activated.head(10)

,from,to,neg_mw,pos_mw
0,2016-01-01 00:00:00,2016-01-01 00:15:00,410.975,2.539
1,2016-01-01 00:15:00,2016-01-01 00:30:00,55.032,22.449
2,2016-01-01 00:30:00,2016-01-01 00:45:00,4.382,253.844
3,2016-01-01 00:45:00,2016-01-01 01:00:00,10.581,71.016
4,2016-01-01 01:00:00,2016-01-01 01:15:00,14.533,104.270
5,2016-01-01 01:15:00,2016-01-01 01:30:00,13.878,72.554
6,2016-01-01 01:30:00,2016-01-01 01:45:00,27.014,17.797
7,2016-01-01 01:45:00,2016-01-01 02:00:00,53.210,5.589
8,2016-01-01 02:00:00,2016-01-01 02:15:00,3.614,285.523
9,2016-01-01 02:15:00,2016-01-01 02:30:00,11.799,75.228
